In [2]:
%load_ext autoreload
%autoreload 2

from util_0804_1 import *

import pandas as pd

## 실험 내용
1. 5주문 번들 제작 최적화
2. 번들 내 주문 수에 따른 WALK 배달원의 존재 여부 확인

### 5주문 번들 제작 기존 코드

In [26]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_10.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0), (0.5, -2, 1)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

    bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 2666.7309999999998
1 -1 2587.2619999999997
0 -0.5 2557.297499999999
0.5 -2 2618.0624999999995


### 5주문 번들 제작 기존 코드를 이용해서 모든 데이터셋의 번들 내 주문 수별 WALK의 수 및 누적으로 줄어드는 가중치별 코드 수행 시간 확인

In [11]:
Ks = []
for problem_file in problem_files:
    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    Ks.append(K)

In [3]:
# 기본 데이터셋 추가
data_sizes = [50, 100, 200]
problem_files = []
for data_size in data_sizes:
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_1.json')
    problem_files.append(fr'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K{data_size}_2.json')

# stage 데이터 추가
for file_num in range(1, 19):
    problem_file = fr'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_{file_num}.json'

    problem_files.append(problem_file)

testcase_names = []
min_costs = []
cts = []
process_times = []
for problem_file in problem_files:
    testcase_name = problem_file.split('\\')[-1].split('.')[0]

    ## ------------------- 기본 변수 할당 ---------------------------

    with open(problem_file, 'r') as f:
        prob = json.load(f)

    K = prob['K']

    ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
    ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

    DIST = np.array(prob['DIST'])
    for r in ALL_RIDERS:
        r.T = np.round(DIST/r.speed + r.service_time).astype(int)

    inf = float('inf')

    timelimit = 60

    ## ------------------- 초기 상태 할당 코드 -------------------------

    car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
    bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
    walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

    init_availables = [rider.available_number for rider in ALL_RIDERS]

    all_bundles = []
    for ord in ALL_ORDERS:
        new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
        car_rider.available_number -= 1
        all_bundles.append(new_bundle)

    ## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

    order_comb_possibility = [[True] * K for _ in range(K)]

    for i in range(len(all_bundles)):
        for j in range(i + 1, len(all_bundles)):
            bundle1 = all_bundles[i]
            bundle2 = all_bundles[j]

            order_num1 = bundle1.shop_seq[0]
            order_num2 = bundle2.shop_seq[0]

            ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

            if not ip:
                order_comb_possibility[order_num1][order_num2] = False
                order_comb_possibility[order_num2][order_num1] = False

    optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

    min_init_cost = inf
    min_init_cost_bundles = []
    min_init_cost_rider_availables = []

    weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0), (0.5, -2, 1), (0.5, -1, 0)]
    start_time = time.time()
    temp_process_times = []
    for weight1, weight2, weight3 in weight_grid:
        temp_start_time = time.time()
        bundles, result_rider_availables, cost = get_init_bundle(K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, try_merging_bundles_by_dist_possibles_only, order_comb_possibility, optimized_order_perms, False, 5)

        bundles, result_rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
        cost = sum((bundle.cost for bundle in bundles)) / K

        temp_end_time = time.time()

        temp_process_time = temp_end_time - temp_start_time
        temp_process_times.append(temp_process_time)

        if cost < min_init_cost:
            min_init_cost = cost
            min_init_cost_bundles = bundles
            min_init_cost_rider_availables = result_rider_availables

    used_rider_info = [(v.rider.type, len(v.shop_seq)) for v in min_init_cost_bundles]

    ct = [[0, 0] for _ in range(6)] # ct[order_c] = [walk_c, non_walk_c]

    for rider_type, order_c in used_rider_info:
        if rider_type == 'WALK':
            ct[order_c][0] += 1
        else:
            ct[order_c][1] += 1

    print(testcase_name, sum(temp_process_times))

    cts.append(ct)
    min_costs.append(min_init_cost)
    testcase_names.append(testcase_name)
    process_times.append(temp_process_times)

TEST_K50_1 1.5587682723999023
TEST_K50_2 1.2418484687805176
TEST_K100_1 6.021928548812866
TEST_K100_2 8.68365478515625
TEST_K200_1 33.15249228477478
TEST_K200_2 21.449501037597656
STAGE1_1 10.54970908164978
STAGE1_2 11.006381511688232
STAGE1_3 24.965724229812622
STAGE1_4 40.6125762462616
STAGE1_5 67.35611176490784
STAGE1_6 42.1063187122345
STAGE1_7 7.738513469696045
STAGE1_8 15.36012053489685
STAGE1_9 28.460707187652588
STAGE1_10 22.87865424156189
STAGE1_11 58.760058879852295
STAGE1_12 37.87260890007019
STAGE1_13 1.5764548778533936
STAGE1_14 5.578274726867676
STAGE1_15 9.834912538528442
STAGE1_16 25.6459321975708
STAGE1_17 13.796545028686523
STAGE1_18 33.55729413032532


In [13]:
for i, v in enumerate(process_times):
    print(testcase_names[i], Ks[i])
    print(v)

TEST_K50_1 50
[0.7245888710021973, 0.30806493759155273, 0.3624305725097656, 0.14768362045288086, 0.01600027084350586]
TEST_K50_2 50
[0.40526437759399414, 0.5196318626403809, 0.16191792488098145, 0.07333159446716309, 0.08170270919799805]
TEST_K100_1 100
[2.226841449737549, 1.405979871749878, 1.135972499847412, 0.8255264759063721, 0.4276082515716553]
TEST_K100_2 100
[2.4961657524108887, 2.647519111633301, 2.0954837799072266, 0.7767367362976074, 0.6677494049072266]
TEST_K200_1 200
[12.561753749847412, 7.801456689834595, 6.295687437057495, 3.616241455078125, 2.8773529529571533]
TEST_K200_2 200
[5.931367635726929, 5.06000542640686, 3.441960573196411, 4.069316625595093, 2.9468507766723633]
STAGE1_1 100
[3.3290326595306396, 2.592778205871582, 2.798473596572876, 1.212083339691162, 0.6173412799835205]
STAGE1_2 100
[5.141759157180786, 3.2448909282684326, 1.2706012725830078, 0.7748160362243652, 0.5743141174316406]
STAGE1_3 200
[9.150849103927612, 6.482074499130249, 2.943791389465332, 3.7896857261

In [15]:
for i, v in enumerate(cts):
    print(testcase_names[i], Ks[i])
    print(v)

TEST_K50_1 50
[[0, 0], [5, 1], [0, 7], [0, 6], [0, 3], [0, 0]]
TEST_K50_2 50
[[0, 0], [5, 3], [0, 12], [0, 6], [0, 0], [0, 0]]
TEST_K100_1 100
[[0, 0], [2, 5], [0, 14], [0, 19], [0, 2], [0, 0]]
TEST_K100_2 100
[[0, 0], [6, 5], [0, 15], [0, 17], [0, 2], [0, 0]]
TEST_K200_1 200
[[0, 0], [13, 5], [1, 19], [0, 30], [0, 13], [0, 0]]
TEST_K200_2 200
[[0, 0], [10, 2], [0, 23], [0, 34], [0, 10], [0, 0]]
STAGE1_1 100
[[0, 0], [0, 7], [0, 18], [0, 12], [0, 4], [0, 1]]
STAGE1_2 100
[[0, 0], [0, 3], [0, 4], [0, 9], [0, 13], [0, 2]]
STAGE1_3 200
[[0, 0], [8, 0], [6, 13], [0, 31], [0, 14], [0, 1]]
STAGE1_4 200
[[0, 0], [0, 0], [1, 4], [0, 13], [0, 29], [0, 7]]
STAGE1_5 300
[[0, 0], [5, 0], [5, 14], [0, 34], [0, 35], [0, 3]]
STAGE1_6 300
[[0, 0], [0, 0], [2, 7], [1, 18], [0, 25], [0, 25]]
STAGE1_7 100
[[0, 0], [0, 3], [0, 15], [0, 13], [0, 7], [0, 0]]
STAGE1_8 100
[[0, 0], [0, 3], [0, 3], [0, 6], [0, 12], [0, 5]]
STAGE1_9 200
[[0, 0], [6, 0], [2, 18], [0, 38], [0, 10], [0, 0]]
STAGE1_10 200
[[0, 0], 

### 5주문 번들 제작 최적화

In [5]:
problem_file = r'C:\Users\hsh80\Desktop\LG CNS\stage1_problems\STAGE1_8.json'
# problem_file = r'C:\Users\hsh80\Desktop\LG CNS\alg_test_problems_20240429\TEST_K50_1.json'

## ------------------- 기본 변수 할당 ---------------------------

with open(problem_file, 'r') as f:
    prob = json.load(f)

K = prob['K']

ALL_ORDERS = [Order(order_info) for order_info in prob['ORDERS']]
ALL_RIDERS = [Rider(rider_info) for rider_info in prob['RIDERS']]

DIST = np.array(prob['DIST'])
for r in ALL_RIDERS:
    r.T = np.round(DIST/r.speed + r.service_time).astype(int)

inf = float('inf')

timelimit = 60

## ------------------- 초기 상태 할당 코드 -------------------------

car_rider = [rider for rider in ALL_RIDERS if rider.type == 'CAR'][0]
bike_rider = [rider for rider in ALL_RIDERS if rider.type == 'BIKE'][0]
walk_rider = [rider for rider in ALL_RIDERS if rider.type == 'WALK'][0]

init_availables = [rider.available_number for rider in ALL_RIDERS]

all_bundles = []
for ord in ALL_ORDERS:
    new_bundle = Bundle(ALL_ORDERS, car_rider, [ord.id], [ord.id], ord.volume, DIST[ord.id, ord.id+K])
    car_rider.available_number -= 1
    all_bundles.append(new_bundle)

## ------------------  주문 조합 가능성 행렬 제작하기 -----------------------------

order_comb_possibility = [[True] * K for _ in range(K)]

for i in range(len(all_bundles)):
    for j in range(i + 1, len(all_bundles)):
        bundle1 = all_bundles[i]
        bundle2 = all_bundles[j]

        order_num1 = bundle1.shop_seq[0]
        order_num2 = bundle2.shop_seq[0]

        ip = try_merging_bundles_by_dist(K, DIST, ALL_ORDERS, ALL_RIDERS, bundle1, bundle2)

        if not ip:
            order_comb_possibility[order_num1][order_num2] = False
            order_comb_possibility[order_num2][order_num1] = False

optimized_order_perms = [dict(), dict(), dict()] # optimized_order_perms[rider_i] = {orders_sorted: 최적 번들}

min_init_cost = inf
min_init_cost_bundles = []
min_init_cost_rider_availables = []

weight_grid = [(1, -3, 0), (1, -1, 0), (0, -0.5, 0)]
temp_process_time = -1
for weight1, weight2, weight3 in weight_grid:
    bundles, rider_availables, cost = get_init_bundle_5_order_bundle_prefered(
        K, ALL_RIDERS, ALL_ORDERS, DIST, init_availables, weight1, weight2, weight3, order_comb_possibility, optimized_order_perms, False)

    bundles, rider_availables = reassign_riders(K, ALL_ORDERS, ALL_RIDERS, DIST, init_availables, bundles)
    cost = sum((bundle.cost for bundle in bundles)) / K

    if cost < min_init_cost:
        min_init_cost = cost
        min_init_cost_bundles = bundles
        min_init_cost_rider_availables = result_rider_availables

    print(weight1, weight2, cost)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
1 -3 3287.172
1 -1 3268.4
0 -0.5 3477.1930000000007


> 일단 추가적인 개선이 없었음